In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


import datasets, and perform semantic analysis on each comments

In [ ]:
import pandas as pd

csv_path = '/content/drive/MyDrive/_work/naist-internship/chatgpt_ai_sentiment/chatgpt_comments.csv'
output_csv_path = '/content/drive/MyDrive/_work/naist-internship/chatgpt_ai_sentiment/chatgpt_comments_with_sentiment.csv'
df = pd.read_csv(csv_path)

In [ ]:
# prompt: loop through each row in the dataframe imported from csv, classify using the twitter roberta sentiment model, and append the percentage of each class into the dataframe, and export all the results into a new csv

from tqdm.notebook import tqdm

# Loop through each row in the dataframe, classify using the model, and append the results
for index, row in tqdm(df.iterrows(), desc="Performing sentiment analysis on comment #"):
    # skip if the text is too long by 500 characters (model doesn't support long text)
    if len(row['post_text']) > 500:
        continue

    # Tokenize the input text
    inputs = tokenizer(row['post_text'], return_tensors="pt")

    encoded_input = tokenizer(row['post_text'], return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    # print(scores)

    # Append the scores to the dataframe
    df.at[index, 'negative'] = scores[0]
    df.at[index, 'neutral'] = scores[1]
    df.at[index, 'positive'] = scores[2]

# Export the results to a new CSV file
df.to_csv(output_csv_path, index=False)


Performing sentiment analysis on comment #: 0it [00:00, ?it/s]